# Zadanie 1
### Autorzy: Wojciech Błyskal 247632, Magdalena Ożarek 247752

## Część 2
### Uzyskaj dostęp do danych nt. COVID-19 z poziomu Pythona.

In [1]:
import os
import pandas as pd
import numpy as np
from google.cloud import bigquery
from functools import reduce

### 2.5. Uruchom klienta BigQuery z poziomu Pythona.

In [2]:
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
#       "C:/Users/ozare/Desktop/BigData/bigdata-zadania-e945fab257f4.json"
# ) # lokalizacja pobranego klucza z punktu 1.4.
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
        "C:/Users/wojte/Documents/studia/semestr 6/BigData/"
        "cool-bay-452611-b5-3811a64fd49a.json"
) # lokalizacja pobranego klucza z punktu 1.4.
client = bigquery.Client() 

### 2.6. Pobierz dane do obiektu DataFrame.

In [3]:
query = ('select * from '
        'bigquery-public-data.covid19_open_data.covid19_open_data limit 10')
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()

## Część 3

### 3.1. Sprawdź, ile jest zapisanych wierszy z danymi.

In [4]:
query = ('SELECT COUNT(*) AS total_rows '
        'FROM bigquery-public-data.covid19_open_data.covid19_open_data')
result = client.query(query).result()
row_count = list(result)[0]["total_rows"]

print(f"Total rows: {row_count}")

Total rows: 22756333


### 3.2. Sprawdź, ile krajów jest uwzględnionych w danych.

In [5]:
query = ('SELECT COUNT(DISTINCT country_code) AS total_rows '
        'FROM bigquery-public-data.covid19_open_data.covid19_open_data')
result = client.query(query).result()
country_count = list(result)[0]["total_rows"]

print(f"Total countries: {country_count}")

Total countries: 246


#### Wnioski: Są różne definicje krajów, ale w uproszczeniu można przyjąć, że jest ich około 200. Zwracanych jest 246 kodów państw, co znaczy, że w jakiś sposób te państwa zostały podzielone na więcej (przez liczenie terytoriów zależnych oddzielnie)

### 3.3. Sprawdź, w jaki sposób zapisywane są dzienne informacje dla krajów.

In [6]:
query = """
        SELECT *
        FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
        WHERE country_name LIKE 'Morocco' AND date = '2021-10-06'
        ORDER BY location_key
"""

df = client.query_and_wait(query).to_dataframe()
df[['location_key', 'date', 'country_name', 'new_confirmed',
        'new_deceased']]

query = """
        SELECT *
        FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
        WHERE country_name LIKE 'France' AND date = '2021-10-06'
        ORDER BY location_key
"""

df = client.query_and_wait(query).to_dataframe()
df[['location_key', 'date', 'country_name', 'new_confirmed',
        'new_deceased']]

query = """
        SELECT *
        FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
        WHERE country_name LIKE 'Guatemala' AND date = '2021-10-06'
        ORDER BY location_key
"""

df = client.query_and_wait(query).to_dataframe()
df[['location_key', 'date', 'country_name', 'new_confirmed',
        'new_deceased', 'average_temperature_celsius']]


,location_key,date,country_name,new_confirmed,new_deceased,average_temperature_celsius
0,GT,2021-10-06,Guatemala,2610,51,23.677778
1,GT_AV,2021-10-06,Guatemala,<NA>,<NA>,24.372222
2,GT_BV,2021-10-06,Guatemala,<NA>,<NA>,24.600000
3,GT_CM,2021-10-06,Guatemala,<NA>,<NA>,25.233333
4,GT_CQ,2021-10-06,Guatemala,<NA>,<NA>,26.027778
5,GT_ES,2021-10-06,Guatemala,<NA>,<NA>,24.894444
6,GT_GU,2021-10-06,Guatemala,<NA>,<NA>,24.894444
7,GT_HU,2021-10-06,Guatemala,<NA>,<NA>,23.100000
8,GT_IZ,2021-10-06,Guatemala,<NA>,<NA>,26.200000
9,GT_JA,2021-10-06,Guatemala,<NA>,<NA>,26.316667


#### Wnioski: Są państwa, np. Maroko, Tunezja, Samoa, gdzie z danego dnia otrzymujemy tylko 1 rekord jako wynik, a są też państwa, np. Polska, Francja, USA, gdzie zwracane jest wiele rekordów. Po przejrzeniu danych okazuje się, iż wynika to z podziału danych na poszczególne regiony- kolumna location_key jest zbudowana następująco: pierwsze dwie litery to kod kraju, kolejne kilka liter to region, po czym następuje ciąg cyfr, który najprawdopodobniej oznacza rozróżnienie na jeszcze mniejsze regiony (np. na departamenty we Francji). Są też kraje, np. Gwatemala, która pomimo posiadania oficjalnego podziału na regiony, dla niektórych danych rozróżnia je między poszczególne regiony, a dla innych są jedynie wartości zbiorcze, np. dane o pogodzie są rozróżnione między regionami, a o zachorowaniach podano jedynie liczby zbiorczo dla całego kraju.

### 3.4. Sprawdź, w jaki sposób zapisywane są wartości liczbowe.

In [7]:
query = """
        SELECT *
        FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
        WHERE country_name LIKE 'France' AND date = '2020-10-06'
                AND new_deceased < 0
        ORDER BY location_key
"""

df = client.query_and_wait(query).to_dataframe()
df[['location_key', 'date', 'country_name', 'new_confirmed',
        'new_deceased']]

query = ('select * from '
        'bigquery-public-data.covid19_open_data.covid19_open_data limit 20')
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()

numeric_dtypes = df.select_dtypes(include=["number"]).dtypes
numeric_dtypes

non_numeric_dtypes = df.select_dtypes(exclude=["number"]).dtypes
non_numeric_dtypes

df[['new_tested', 'population_largest_city',
        'population_clustered', 'human_capital_index', 'area_rural_sq_km',
        'area_urban_sq_km', 'life_expectancy']]
df[['adult_male_mortality_rate',
        'adult_female_mortality_rate', 'pollution_mortality_rate',
        'comorbidity_mortality_rate', 'mobility_retail_and_recreation']]
df[['mobility_grocery_and_pharmacy', 'mobility_parks',
        'mobility_transit_stations', 'mobility_workplaces',
        'mobility_residential', 'age_bin_0', 'location_geometry']]


,mobility_grocery_and_pharmacy,mobility_parks,mobility_transit_stations,mobility_workplaces,mobility_residential,age_bin_0,location_geometry
0,33,-4,1,8,4,0-9,POINT(-64 -34)
1,-17,-67,-60,-58,22,0-9,POINT(-64 -34)
2,9,-44,-38,-21,15,0-9,POINT(-53 -14)
3,39,-1,-7,4,6,0-9,POINT(-53 -14)
4,25,-36,-16,-3,8,0-9,POINT(-53 -14)
5,35,3,24,10,4,0-9,POINT(-73.25 4)
6,22,-8,-12,-16,12,0-9,POINT(-73.25 4)
7,35,2,2,-12,7,0-9,POINT(-73.25 4)
8,-66,-79,-81,-57,26,0-9,POINT(-73.25 4)
9,26,-1,7,-3,7,0-9,POINT(-73.25 4)


#### Wnioski: Wartości liczbowe są na ogół przechowywane jako int64 lub float64. Wyjątkiem są kolumny, które są przechowywane jako obiekt. Niektóre z kolumn, które przechowują obiekty, zawierają dane na pozór liczbowe. Są też miejsca, gdzie są podane wartości ujemne, które muszą mieć pewne specjalne znaczenie.

### 3.5. Sprawdź, jaki przedział czasowy jest uwzględniony w danych. Dodatkowo porównaj przedziały czasowe dla przypadków nowych zachorowań, nowych śmierci oraz nowych zaszczepionych osób w danych.

In [8]:
query = ('SELECT MIN(date) AS start_date, MAX(date) AS end_date '
        'FROM bigquery-public-data.covid19_open_data.covid19_open_data')
result = client.query(query).result()
row = next(result)
start_date = row.start_date
end_date = row.end_date

print("Whole data period: " + str(start_date) + "/" + str(end_date))

query = ('SELECT MIN(date) AS start_date, MAX(date) AS end_date '
        'FROM bigquery-public-data.covid19_open_data.covid19_open_data '
        'WHERE new_confirmed IS NOT NULL')
result = client.query(query).result()
row = next(result)
start_date = row.start_date
end_date = row.end_date

print("Data period of new confirmed cases: " + str(start_date) + "/"
      + str(end_date))

query = ('SELECT MIN(date) AS start_date, MAX(date) AS end_date '
        'FROM bigquery-public-data.covid19_open_data.covid19_open_data '
        'WHERE new_deceased IS NOT NULL')
result = client.query(query).result()
row = next(result)
start_date = row.start_date
end_date = row.end_date

print("Data period of new deceased cases: " + str(start_date) + "/"
      + str(end_date))

query = ('SELECT MIN(date) AS start_date, MAX(date) AS end_date '
        'FROM bigquery-public-data.covid19_open_data.covid19_open_data '
        'WHERE new_persons_vaccinated IS NOT NULL')
result = client.query(query).result()
row = next(result)
start_date = row.start_date
end_date = row.end_date

print("Data period of new vaccinated persons cases: " + str(start_date) + "/"
      + str(end_date))

query = ('SELECT new_confirmed, new_deceased, new_persons_vaccinated '
        'FROM bigquery-public-data.covid19_open_data.covid19_open_data '
        'WHERE date = "2022-09-17"')
result = client.query(query).result()
row = next(result)
print(str(row.new_confirmed) + ", " + str(row.new_deceased) + ", "
      + str(row.new_persons_vaccinated))

Whole data period: 2020-01-01/2022-09-17
Data period of new confirmed cases: 2020-01-01/2022-09-16
Data period of new deceased cases: 2020-01-01/2022-09-16
Data period of new vaccinated persons cases: 2020-03-12/2022-09-16
None, None, None


#### Wnioski: Od pierwszego dnia zbierania danych odnotowywane są przypadki zarażenia i zgonu, ale szczepienia zaczynają się z dopiero w marcu, co prawdopodobnie wynika z niepodawania jej wcześniej ludziom. Ostatniego dnia nie było żadnych danych na temat nowych zakażeń, śmierci lub szczepień.

### 3.6. Sprawdź więcej informacji (co najmniej 5 różnych) o danych dotyczących COVID-19. W tym celu nie wykonuj żadnych dodatkowych obliczeń.

#### 1.

#### 2.

#### 3.

#### 4.

#### 5.

## Część 4

### Przeanalizuj poniższe przypadki. Zastanów się, jakie dane potrzebujesz do każdego z nich, a następnie zapisz je w osobnych, jak najprostszych obiektach DataFrame. Na tym etapie nie przetwarzaj ich. Zadbaj o czystość danych, m.in. usuń niepotrzebne puste wartości i duplikaty, zunifikuj sposób podawania informacji, zaproponuj sposób naprawy błędnych danych. Gotowe dane z obiektów DataFrame zapisz w osobnych plikach CSV.

### 4.1 Chcemy posiadać podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i potencjalnie przyszłościowe do dalszego przetwarzania.

In [9]:
query = ('SELECT DISTINCT(country_name), iso_3166_1_alpha_2, iso_3166_1_alpha_3, '
        'population, population_male, population_female, population_rural, '
        'population_urban, population_density, human_development_index, '
        'gdp_usd, gdp_per_capita_usd, population_largest_city, '
        'life_expectancy, human_capital_index, area_sq_km, date '
        'FROM bigquery-public-data.covid19_open_data.covid19_open_data '
)

query_job = client.query(query)
query_result = query_job.result()
df_1 = query_result.to_dataframe()

df_1

df_1 = df_1.replace({None: np.nan, pd.NA: np.nan})

total_missing = df_1.isna().sum().sum()
if total_missing > 0:
        print("Number of missing values in each column of the DataFrame:")
        print(df_1.isna().sum())
        print(f"Total of missing values in the DataFrame: {total_missing}")
        df_missing = df_1[df_1.isna().any(axis=1)]
        print("Rows with missing values:")
        df_missing

df_1 = df_1.dropna()

print("Done")

#Usuwanie pustych rekordów
total_missing = df_1.isna().sum().sum()
if total_missing > 0:
        print("Number of missing values in each column of the DataFrame:")
        print(df_1.isna().sum())
        print(f"Total of missing values in the DataFrame: {total_missing}")
        df_missing = df_1[df_1.isna().any(axis=1)]
        print("Rows with missing values:")
        df_missing

# Usuwanie duplikatów
num_duplicates = df_1.duplicated().sum()
print(f"Number of duplicates: {num_duplicates}")

df_duplicates = df_1[df_1.duplicated()]
if num_duplicates > 0:
        print("Rows with duplicates:")
        df_duplicates
        df_1 = df_1.drop_duplicates()
        df_1 = df_1.reset_index(drop=True)
        df_1

# Konwersja typu w kolumnie life_expectancy
query = ('SELECT life_expectancy '
        'FROM bigquery-public-data.covid19_open_data.covid19_open_data '
         'WHERE life_expectancy NOT LIKE "__.%" OR life_expectancy NOT LIKE "___.%"'
        'GROUP BY life_expectancy '
)

query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df_1

df_1['life_expectancy'] = df_1['life_expectancy'].astype(float)
print("Life expectancy type: ", df_1['life_expectancy'].dtype)

# Zapis do pliku
df_1.to_csv("4_1.csv", index=False)

Number of missing values in each column of the DataFrame:
country_name                      0
iso_3166_1_alpha_2              991
iso_3166_1_alpha_3              991
population                   511356
population_male             4520942
population_female           4520942
population_rural           21735603
population_urban           21735603
population_density         20765414
human_development_index    16277175
gdp_usd                    21577043
gdp_per_capita_usd         21584971
population_largest_city    21796054
life_expectancy            18479177
human_capital_index        21792090
area_sq_km                  2780746
date                              0
dtype: int64
Total of missing values in the DataFrame: 198079098
Rows with missing values:
Done
Number of duplicates: 0
Life expectancy type:  float64


### 4.2. Chcemy wygenerować statystyki dotyczące zachorowań na COVID-19 na całym świecie.

In [10]:
query = ('SELECT date, new_confirmed, cumulative_confirmed, new_confirmed_male, '
        'new_confirmed_female, cumulative_confirmed_male, '
        'cumulative_confirmed_female, new_confirmed_age_0, '
        'new_confirmed_age_1, new_confirmed_age_2, new_confirmed_age_3, '
        'new_confirmed_age_4, new_confirmed_age_5, new_confirmed_age_6, '
        'new_confirmed_age_7, new_confirmed_age_8, new_confirmed_age_9, '
        'cumulative_confirmed_age_0, cumulative_confirmed_age_1, '
        'cumulative_confirmed_age_2, cumulative_confirmed_age_3, '
        'cumulative_confirmed_age_4, cumulative_confirmed_age_5, '
        'cumulative_confirmed_age_6, cumulative_confirmed_age_7, '
        'cumulative_confirmed_age_8, cumulative_confirmed_age_9 ' 
        'FROM bigquery-public-data.covid19_open_data.covid19_open_data')
query_job = client.query(query)
query_result = query_job.result()
df_2 = query_result.to_dataframe()

# Usuwanie duplikatów
num_duplicates = df_2.duplicated().sum()
print(f"Number of duplicates: {num_duplicates}")

df_duplicates = df_2[df_2.duplicated()]
if num_duplicates > 0:
        print("Rows with duplicates:")
        df_duplicates
        df_2 = df_2.drop_duplicates()
        df_2 = df_2.reset_index(drop=True)
        df_2

# Zapis do pliku
df_2.to_csv("4_2.csv", index=False)

Number of duplicates: 13035014
Rows with duplicates:


### 4.3. Chcemy poznać efekty COVID-19 poprzez uwypuklenie problemu śmiertelności ludzi spowodowanej wirusem.

In [11]:
query = ('SELECT date, new_deceased, cumulative_deceased, new_deceased_male, '
        'new_deceased_female, cumulative_deceased_male, '
        'cumulative_deceased_female, '
        'cumulative_deceased_age_0, cumulative_deceased_age_1, '
        'cumulative_deceased_age_2, cumulative_deceased_age_3, '
        'cumulative_deceased_age_4, cumulative_deceased_age_5, '
        'cumulative_deceased_age_6, cumulative_deceased_age_7, '
        'cumulative_deceased_age_8, cumulative_deceased_age_9 '
        'FROM bigquery-public-data.covid19_open_data.covid19_open_data')
query_job = client.query(query)
query_result = query_job.result()
df_3 = query_result.to_dataframe()

# Usuwanie duplikatów
num_duplicates = df_3.duplicated().sum()
print(f"Number of duplicates: {num_duplicates}")

df_duplicates = df_3[df_3.duplicated()]
if num_duplicates > 0:
        print("Rows with duplicates:")
        df_duplicates
        df_3 = df_3.drop_duplicates()
        df_3 = df_3.reset_index(drop=True)
        df_3

# Zapis do pliku
df_3.to_csv("4_3.csv", index=False)

Number of duplicates: 19344918
Rows with duplicates:


### 4.4. Chcemy zaobserwować trendy i zależności dotyczące szczepień na COVID-19.

In [12]:
query = ('SELECT date, new_persons_vaccinated, '
        'cumulative_persons_vaccinated, new_persons_fully_vaccinated, ' 
        'cumulative_persons_fully_vaccinated, '
        'new_vaccine_doses_administered, '
        'cumulative_vaccine_doses_administered '
        'FROM bigquery-public-data.covid19_open_data.covid19_open_data')
query_job = client.query(query)
query_result = query_job.result()
df_4 = query_result.to_dataframe()

# Usuwanie duplikatów
num_duplicates = df_4.duplicated().sum()
print(f"Number of duplicates: {num_duplicates}")

df_duplicates = df_4[df_4.duplicated()]
if num_duplicates > 0:
        print("Rows with duplicates:")
        df_duplicates
        df_4 = df_4.drop_duplicates()
        df_4 = df_4.reset_index(drop=True)
        df_4

# Zapis do pliku
df_4.to_csv("4_4.csv", index=False)

Number of duplicates: 20385043
Rows with duplicates:


### 4.5. Zdefiniuj własny dodatkowy przypadek. 
#### Przypadek: Chcemy wygenerować statystyki dotyczące ogólnej opieki zdrowotnej.

In [13]:
query = ('SELECT date, nurses_per_1000, physicians_per_1000, '
        'health_expenditure_usd, out_of_pocket_health_expenditure_usd '
        'FROM bigquery-public-data.covid19_open_data.covid19_open_data')
query_job = client.query(query)
query_result = query_job.result()
df_5 = query_result.to_dataframe()

# Usuwanie duplikatów
num_duplicates = df_5.duplicated().sum()
print(f"Number of duplicates: {num_duplicates}")

df_duplicates = df_5[df_5.duplicated()]
if num_duplicates > 0:
        print("Rows with duplicates:")
        df_duplicates
        df_5 = df_5.drop_duplicates()
        df_5 = df_5.reset_index(drop=True)
        df_5

# Zapis do pliku
df_5.to_csv("4_5.csv", index=False)

Number of duplicates: 22566061
Rows with duplicates:


## Część 5

### Połącz ze sobą wszystkie dane otrzymane w części 4. Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

In [14]:
df_list = [df_1, df_2, df_3, df_4, df_5]

merged_df_5 = reduce(lambda left, right: pd.merge(left, right, on='date', how='inner'), df_list)

# Zapis do pliku
merged_df_5.to_csv("5.csv", index=False)

MemoryError: Unable to allocate 9.56 GiB for an array with shape (1283214108,) and data type int64

## Część 6

### Połącz ze sobą dane otrzymane w części 5 oraz dane znajdujące się w plikach:

### 6.1. world_countries.csv. Podstawowe dane dotyczące krajów na całym świecie razem z historycznymi danymi na temat ich populacji.

In [ ]:
key_cols = ['iso_3166_1_alpha_3', 'country_name']
non_key_cols = [col for col in df1.columns if col not in key_cols]

query = ('SELECT * FROM world.countries.csv ')
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()

df.rename(columns={"CCA3" : "iso_3166_1_alpha_3", "Country/Territory" : "country_name"})

merged_df_6_1 = df.merge(merged_df_5, on=key_cols, how='left', suffixes=('_old', ''))
merged_df_6_1.drop(columns=[col + '_old' for col in non_key_cols], inplace=True)

### 6.2. GDP.csv Wartości produktu krajowego brutto (w skrócie PKB, ang. GDP - gross domestic product) w krajach i regionach świata. 

In [ ]:
key_cols = ['iso_3166_1_alpha_3', 'country_name']
non_key_cols = [col for col in df1.columns if col not in key_cols]

query = ('SELECT * FROM gdp.csv ')
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()

df.rename(columns={"Country Code" : "iso_3166_1_alpha_3", "Country Name" : "country_name"})

merged_df_6_2 = df.merge(merged_df_6_1, on=key_cols, how='left', suffixes=('_old', ''))
merged_df_6_2.drop(columns=[col + '_old' for col in non_key_cols], inplace=True)

### Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

In [ ]:
merged_df_6_2.to_csv("6_2.csv", index=False)

## Część 7

### Znajdź 3 dodatkowe zbiory danych, które uważasz, że rozwijają w ciekawy sposób temat ostatniej pandemii. Powtórz dla nich analogiczne czynności z części 4, a następnie połącz je z danymi z części 6. Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.